In [1]:
import os
import io
import csv
import sys
import re
from collections import defaultdict
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math


In [2]:
def getData1D(filepath):
    file = open(filepath)
    reader = csv.reader(file)
    data_str = list(reader)
    file.close()
    X = [float(x[0]) for x in data_str]
    return X

def getData4D(filepath):
    file = open(filepath)
    reader = csv.reader(file)
    data_str = list(reader)
    file.close()
    X1 = [float(x[0]) for x in data_str]
    X2 = [float(x[1]) for x in data_str]
    X3 = [float(x[2]) for x in data_str]
    X4 = [float(x[3]) for x in data_str]
    return X1, X2, X3, X4

def getData7D(filepath):
    file = open(filepath)
    reader = csv.reader(file)
    data_str = list(reader)
    file.close()
    X1 = [float(x[0]) for x in data_str]
    X2 = [float(x[1]) for x in data_str]
    X3 = [float(x[2]) for x in data_str]
    X4 = [float(x[3]) for x in data_str]
    X5 = [float(x[4]) for x in data_str]
    X6 = [float(x[5]) for x in data_str]
    X7 = [float(x[6]) for x in data_str]
    return X1, X2, X3, X4, X5, X6, X7

def getData5D(filepath):
    file = open(filepath)
    reader = csv.reader(file)
    data_str = list(reader)
    file.close()
    X1 = [float(x[0]) for x in data_str[1:]]
    X2 = [float(x[1]) for x in data_str[1:]]
    X3 = [float(x[2]) for x in data_str[1:]]
    X4 = [float(x[3]) for x in data_str[1:]]
    rob = [float(x[4]) for x in data_str[1:]]
    return X1, X2, X3, X4, rob

def getData8D(filepath):
    file = open(filepath)
    reader = csv.reader(file)
    data_str = list(reader)
    file.close()
    X1 = [float(x[0]) for x in data_str[1:]]
    X2 = [float(x[1]) for x in data_str[1:]]
    X3 = [float(x[2]) for x in data_str[1:]]
    X4 = [float(x[3]) for x in data_str[1:]]
    X5 = [float(x[4]) for x in data_str[1:]]
    X6 = [float(x[5]) for x in data_str[1:]]
    X7 = [float(x[6]) for x in data_str[1:]]
    rob = [float(x[7]) for x in data_str[1:]]
    return X1, X2, X3, X4, X5, X6, X7, rob



# Parameters (set it up YO-self)

In [3]:
scenario_names = ['LVM1', 'LVM2', 'LVDAD', 'LVD', 'POV_UnprotectedLeft', 'OppositeVehicleRunningRedLight']
scenario = scenario_names[5]
print("SCENARIO:", scenario)
#
res_path = "../../results/"+scenario
dir_list = next(os.walk(res_path))[0]
# print (dir_list)
#
scenario_folders = [os.path.join(res_path, f) for f in next(os.walk(dir_list))[1] if not os.path.isdir(f)]
print('folders to analyze ', scenario_folders)
#
robustness_fn = 'f_history.csv'
params_fn = 'x_history.csv'
#

lat_flag = 1
all_flag = 0

if scenario == 'OppositeVehicleRunningRedLight':
    lat_flag = 1
elif scenario == 'POV_UnprotectedLeft':
#     all_flag = 1
    lat_flag = 0
else:
    lat_flag = 0

X1_name = 'alpha_lon_accel_max'
X2_name = 'alpha_lon_brake_max'
X3_name = 'alpha_lon_brake_min'
X4_name = 'response_time'
X5_name = 'alpha_lat_accel_max'
X6_name = 'alpha_lat_brake_min'
X7_name = 'lateral_fluctuation_margin'
X8_name = 'alpha_lon_brake_min_correct'
rob = 'robustness'


if lat_flag:
    search_names = [X1_name, X2_name, X3_name, X4_name, X5_name, X6_name, X7_name, rob]
else:
    search_names = [X1_name, X2_name, X3_name, X4_name, rob]
    
if all_flag:
    search_names = [X1_name, X2_name, X3_name, X4_name, X5_name, X6_name, X7_name, X8_name, rob]

print('search names', search_names)

SCENARIO: OppositeVehicleRunningRedLight
folders to analyze  ['../../results/OppositeVehicleRunningRedLight/results-Rss_OppositeVehicleRunningRedLight-2020-01-17-14-12-53', '../../results/OppositeVehicleRunningRedLight/results-Rss_OppositeVehicleRunningRedLight-2020-01-17-18-27-26']
search names ['alpha_lon_accel_max', 'alpha_lon_brake_max', 'alpha_lon_brake_min', 'response_time', 'alpha_lat_accel_max', 'alpha_lat_brake_min', 'lateral_fluctuation_margin', 'robustness']


# Creating a dictionary to find duplicated data points


In [4]:
## creating a dictionary to find duplicated data points
X1 = []
X2 = []
X3 = []
X4 = []
X5 = []
X6 = []
X7 = []
robustness = []
data_dict = defaultdict(list)

for folder in scenario_folders:
    print("reading folder ", folder)
    rob_fn = os.path.join(folder, robustness_fn)
    par_fn = os.path.join(folder, params_fn)
    
    robustness = getData1D(rob_fn)
    data_size = len(robustness)
    
    
    if lat_flag == 1:
        X1,X2,X3,X4,X5,X6,X7 = getData7D(par_fn)
    else:
        X1,X2,X3,X4 = getData4D(par_fn)
    
    print("data size ", len(robustness))
    
    for i in range(data_size):
        
        if lat_flag == 0:
            str_keys = [str(X1[i]), str(X2[i]), str(X3[i]), str(X4[i])]
            key = '_'.join(str_keys)

            if data_dict[key]:
                data_dict[key][rob].append(robustness[i])
                data_dict[key][search_names[0]].append(X1[i])
                data_dict[key][search_names[1]].append(X2[i])
                data_dict[key][search_names[2]].append(X3[i])
                data_dict[key][search_names[3]].append(X4[i])
                print('repeated params:', key, ', robustness:', data_dict[key]['robustness'])

            else: 
                data_dict[key] = {search_names[0]:[], 
                                  search_names[1]:[], 
                                  search_names[2]:[], 
                                  search_names[3]:[], 
                                  'robustness':[]}
                data_dict[key][rob].append(robustness[i])
                data_dict[key][search_names[0]].append(X1[i])
                data_dict[key][search_names[1]].append(X2[i])
                data_dict[key][search_names[2]].append(X3[i])
                data_dict[key][search_names[3]].append(X4[i])

        else:
            str_keys = [str(X1[i]), str(X2[i]), str(X3[i]), str(X4[i]), str(X5[i]), str(X6[i]), str(X7[i])]
            key = '_'.join(str_keys)

            if data_dict[key]:
                data_dict[key][rob].append(robustness[i])
                data_dict[key][search_names[0]].append(X1[i])
                data_dict[key][search_names[1]].append(X2[i])
                data_dict[key][search_names[2]].append(X3[i])
                data_dict[key][search_names[3]].append(X4[i])
                data_dict[key][search_names[4]].append(X5[i])
                data_dict[key][search_names[5]].append(X6[i])
                data_dict[key][search_names[6]].append(X7[i])
                
                print('repeated params:', key, ', robustness:', data_dict[key]['robustness'])

            else: 
                data_dict[key] = {search_names[0]:[], 
                                  search_names[1]:[],
                                  search_names[2]:[], 
                                  search_names[3]:[], 
                                  search_names[4]:[], 
                                  search_names[5]:[], 
                                  search_names[6]:[], 
                                  'robustness':[]}
                
                data_dict[key][rob].append(robustness[i])
                data_dict[key][search_names[0]].append(X1[i])
                data_dict[key][search_names[1]].append(X2[i])
                data_dict[key][search_names[2]].append(X3[i])
                data_dict[key][search_names[3]].append(X4[i])
                data_dict[key][search_names[4]].append(X5[i])
                data_dict[key][search_names[5]].append(X6[i])
                data_dict[key][search_names[6]].append(X7[i])
                
    print("final data size ", len(data_dict.keys()))
  

reading folder  ../../results/OppositeVehicleRunningRedLight/results-Rss_OppositeVehicleRunningRedLight-2020-01-17-14-12-53
data size  3001
final data size  3001
reading folder  ../../results/OppositeVehicleRunningRedLight/results-Rss_OppositeVehicleRunningRedLight-2020-01-17-18-27-26
data size  3001
repeated params: 3.5_6.0_3.5_1.0_0.1_0.1_0.1 , robustness: [35.42826989797114, 37.87884679556028]
final data size  6001


# Save data to csv file 

In [5]:
### Save data to csv file 
with open(os.path.join(res_path, scenario + '_combined.csv'), 'w') as f:
    
#     if lat_flag == 0:
#         fieldnames = [X1_name, X2_name, X3_name, X4_name, 'robustness']
#     else:
#         fieldnames = [X1_name, X2_name, X3_name, X4_name, 'robustness']

    writer = csv.writer(f)
    writer.writerow([rob]+search_names[:-1])
    
    for entry in data_dict:
        if lat_flag == 0:
            vals = [data_dict[entry]['robustness'][0],
                    data_dict[entry][search_names[0]][0], 
                    data_dict[entry][search_names[1]][0], 
                    data_dict[entry][search_names[2]][0], 
                    data_dict[entry][search_names[3]][0]]
        else:
            vals = [data_dict[entry]['robustness'][0],
                    data_dict[entry][search_names[0]][0], 
                    data_dict[entry][search_names[1]][0], 
                    data_dict[entry][search_names[2]][0], 
                    data_dict[entry][search_names[3]][0], 
                    data_dict[entry][search_names[4]][0], 
                    data_dict[entry][search_names[5]][0], 
                    data_dict[entry][search_names[6]][0]]
        writer.writerow(vals)     

# Read data from a csv to pandas data frame

In [6]:
# Read data from a csv to pandas data frame
data = pd.read_csv(os.path.join(res_path, scenario + '_combined.csv'), index_col=False)

# sort values by robustness
data = data.sort_values(by=['robustness'])
data.to_csv(os.path.join(res_path, scenario + '_combined.csv'), index=False)

rob_min = data['robustness'].iloc[0]
rob_max = data['robustness'].iloc[-1]

# create bins for data binning
bins = range(0, math.ceil(rob_max)+1, 1)
print(bins)

range(0, 39)


# create bins based on robustness values

In [7]:
## separate data into bins and extract min max
min_bin_params = []
max_bin_params = []
desc_vals = []

for i in bins:
#     print(i)
    if i == 0:
        beg = i
        end = i
        data_aux = data[data['robustness'] <= end]
    else:
        beg = i-1
        end = i
        data_aux = data[data['robustness'] <= end]
        data_aux = data_aux[data_aux['robustness'] > beg]

    if len(data_aux) == 0:
        
        desc_vals.append([X1_name] + [0,0,0,0,0,0,0,0] +[i])
        desc_vals.append([X2_name] + [0,0,0,0,0,0,0,0] +[i])
        desc_vals.append([X3_name] + [0,0,0,0,0,0,0,0] +[i])
        desc_vals.append([X4_name] + [0,0,0,0,0,0,0,0] +[i])

    else:  
        
        min_bin_params.append(list(data_aux.iloc[0]))
        max_bin_params.append(list(data_aux.iloc[-1]))

        desc_vals.append([X1_name] + data_aux[X1_name].describe().tolist()+[i])
        desc_vals.append([X2_name] + data_aux[X2_name].describe().tolist()+[i])
        desc_vals.append([X3_name] + data_aux[X3_name].describe().tolist()+[i])
        desc_vals.append([X4_name] + data_aux[X4_name].describe().tolist()+[i]) 
                   

        
### Save results to csv        
with open(os.path.join(res_path, scenario + '_param_bounds.csv'), 'w') as f:
    
    fieldnames = ['name'] + data_aux[X1_name].describe().keys().tolist()+['robustness_bin']
#     print(fieldnames)
    
    writer = csv.writer(f)
    writer.writerow(fieldnames)
    
    for row in desc_vals:
        writer.writerow(row) 